In [ ]:
%matplotlib widget
from matplotlib import pyplot as plt
import numpy as np
import scipy.io as scio
import netCDF4
import sys
sys.path.append('./modules/')
from emit_tools import emit_xarray
import geo_matching_utils
#import match_extent

import xarray as xr
import rioxarray
import holoviews as hv
import hvplot.xarray
import rasterio
from rasterio.plot import show
from osgeo import osr, gdal
from scipy import interpolate

from spectral.io import envi

### Option 1: Resave EMIT and use gdal warp

In [ ]:
#Save EMIT as orthorectified ENVI file
emit_fp = '../data/EMIT_L2A_RFL_001_20230728T214106_2320914_002.nc'
geo_matching_utils.reformat(input_netcdf=emit_fp,\
    output_dir='../data/',output_type='ENVI',interleave='BIL',\
                            overwrite=False,orthorectify=True)


In [ ]:
emit_fp = '../data/EMIT_L2A_RFL_001_20230728T214106_2320914_002.nc'

In [ ]:
#Use gdalwarp to warp ECOSTRESS file to EMIT geo
emit_fp_ortho = emit_fp.replace('.nc','_reflectance')
es_fp = '../data/ECOv002_L2T_LSTE_28691_004_11SLT_20230728T214058_0710_01_LST.tif'
es_fp_ortho = es_fp.replace('.tif','_emitProj.tif')
geo_matching_utils.match_extent(source_file='../data/ECOv002_L2T_LSTE_28691_004_11SLT_20230728T214058_0710_01_LST.tif',\
    match_file=emit_fp_ortho, dst_file = es_fp_ortho,resample_method='near',\
    output_format = 'GTiff',srcnodata='-9999',dstnodata='-9999')

In [ ]:
#Read in ortho'd files
emit_mm = envi.open(emit_fp_ortho + '.hdr').open_memmap() #Acts like a numpy array
es_rio = rasterio.open(es_fp_ortho)

In [ ]:
geotiff_da = rioxarray.open_rasterio(es_fp)
geotiff_ds = geotiff_da.to_dataset('band')
# Rename the variable to a more useful name
geotiff_ds = geotiff_ds.rename({1: 'LSTE'})

In [ ]:
geotiff_np = np.array(geotiff_ds.LSTE)

In [ ]:
plt.figure()
plt.imshow(geotiff_np)
plt.imshow(emit_mm[:,:,80])

In [ ]:
geotiff_ds.x.values

In [ ]:
geotiff_ds.y.values

In [ ]:
emit_hdr = envi.open(emit_fp_ortho + '.hdr')

In [ ]:
emit_hdr.metadata

In [ ]:
es_ds.GetProjectionRef()

In [ ]:
es_ds

In [ ]:
es_ds = gdal.Open(es_fp_ortho)
es_np = np.array(es_ds.GetRasterBand(1).ReadAsArray())

In [ ]:
es_np

In [ ]:
plt.figure()
plt.imshow(es_np)

In [ ]:
es_np.shape

In [ ]:
emit_mm.shape

In [ ]:
plt.figure()
plt.imshow(emit_mm[:,:,80])
plt.colorbar()
plt.clim((0,1))

In [ ]:
es_np

In [ ]:
plt.figure()
plt.imshow(es_np)

### Read in EMIT data

In [ ]:
# Read in with xarray, using emit_xarry module helper tools (useful for orthorectification)
fp = '../data/EMIT_L2A_RFL_001_20230728T214106_2320914_002.nc'
ds_geo = emit_xarray(fp, ortho=True)


In [ ]:
ds_geo.attrs['spatial_ref']

In [ ]:
ds_geo.geotransform

In [ ]:
ds_geo.sel(wavelengths=850, method='nearest').hvplot.image(cmap='viridis', frame_width=500, geo=True, tiles='EsriImagery').opts(
    xlabel=f'{ds_geo.longitude.long_name} ({ds_geo.longitude.units})', ylabel=f'{ds_geo.latitude.long_name} ({ds_geo.latitude.units})')

### ECOSTRESS

In [ ]:
# Read in with xarray
geotiff_path = '../data/ECOv002_L2T_LSTE_28691_004_11SLT_20230728T214058_0710_01_LST.tif'
geotiff_da = rioxarray.open_rasterio(geotiff_path)
geotiff_ds = geotiff_da.to_dataset('band')
# Rename the variable to a more useful name
geotiff_ds = geotiff_ds.rename({1: 'LSTE'})

### Interpolate ECOSTRESS to EMIT data (scipy method)

In [ ]:
def convert_xy_to_latlon(old_cs,new_cs,x_coordinates,y_coordinates):
    # create a transform object to convert between coordinate systems
    transform = osr.CoordinateTransformation(old_cs,new_cs) 
    latlon = np.zeros((len(x_coordinates),3))
    for ii in range(len(x_coordinates)):
        latlon[ii,:] = transform.TransformPoint(x_coordinates[ii],y_coordinates[ii])
    return latlon
        

In [ ]:
geotiff_path = es_fp_ortho

In [ ]:
geotiff_ds.coords['y'].values.shape

In [ ]:
ds = gdal.Open(geotiff_path)

# get the existing coordinate system
old_cs= osr.SpatialReference()
old_cs.ImportFromWkt(ds.GetProjectionRef())

wgs84_wkt = ds_geo.attrs['spatial_ref']
new_cs = osr.SpatialReference()
new_cs .ImportFromWkt(wgs84_wkt)

# create a transform object to convert between coordinate systems
transform = osr.CoordinateTransformation(old_cs,new_cs) 

#get the coordinates in lat long
latlon = convert_xy_to_latlon(old_cs,new_cs,geotiff_ds.coords['x'].values,geotiff_ds.coords['y'].values)

#Assign them in xarray
geotiff_ds = geotiff_ds.assign_coords({'latitude':latlon[:,0],'longitude':latlon[:,1]})


In [ ]:
#The new latlon grid we want things on
latlon_new = np.meshgrid(ds_geo.coords['latitude'].values,ds_geo.coords['longitude'].values,indexing='ij')

In [ ]:
lste_interp = interpolate.interpn((geotiff_ds.coords['latitude'].values,geotiff_ds.coords['longitude'].values),\
                               geotiff_ds.LSTE.to_numpy(),\
                               (latlon_new[0],latlon_new[1]),\
                               bounds_error=False)

In [ ]:
nprfl = ds_geo.reflectance.to_numpy()

In [ ]:
plt.figure()
plt.imshow(lste_interp)
ax = plt.gca()
ax.set_ylim((1750,1250))
ax.set_xlim((300,1000))

In [ ]:
plt.figure()
plt.imshow(lste_interp)
plt.imshow(nprfl[:,:,80])
ax = plt.gca()
ax.set_ylim((1750,1250))
ax.set_xlim((300,1000))

In [ ]:
plt.figure()
plt.imshow(lste_interp)
plt.imshow(nprfl[:,:,80])

In [ ]:
# Can now use lste_interp and nprfl as the two data arrays
# They aren't perfectly matching yet, but enough to get started

### Process ECOSTRESS to enhance thermal anomalies

In [ ]:
# *** Under construction *** #

### Other ways to read in EMIT data

In [ ]:
#emit_data = scio.netcdf.netcdf_file('../data/EMIT_L2A_RFL_001_20230728T214106_2320914_002.nc',mode='r')
emit_data = netCDF4.Dataset('../data/EMIT_L2A_RFL_001_20230728T214106_2320914_002.nc')
emit_data_uncert = netCDF4.Dataset('../data/EMIT_L2A_RFLUNCERT_001_20230728T214106_2320914_002.nc')
emit_data_mask = netCDF4.Dataset('../data/EMIT_L2A_MASK_001_20230728T214106_2320914_002.nc')

In [ ]:
emit_data

In [ ]:
plt.figure()
plt.plot(emit_data['/sensor_band_parameters/wavelengths/'][:])
plt.plot(emit_data.groups['sensor_band_parameters'].variables['wavelengths'][:])

In [ ]:
emit_data.groups['location'].variables.keys()

In [ ]:
emit_data.variables['reflectance'].shape

In [ ]:
plt.figure()
plt.imshow(emit_data.variables['reflectance'][:,:,50])

In [ ]:
emit_data.groups['location'].variables['lon'].shape

In [ ]:
emit_data.groups['location'].variables['glt_x'].shape

In [ ]:
plt.figure()
plt.imshow(emit_data.groups['location'].variables['lon'])

In [ ]:
emit_data_mask['mask'].shape

In [ ]:
plt.figure()
plt.imshow(emit_data_mask['mask'][:,:,7])
plt.colorbar()

### Other ways to read in ECOSTRESS data

In [ ]:
ds = gdal.Open('../data/ECOv002_L2T_LSTE_28691_004_11SLT_20230728T214058_0710_01_LST.tif')

In [ ]:
img = rasterio.open('../data/ECOSTRESS_LA_daytime_summer_LST_2018_2021.tif')
show(img)

In [ ]:
img = rasterio.open('../data/ECOv002_L2T_LSTE_28691_004_11SLT_20230728T214058_0710_01_LST.tif')
show(img)

In [ ]:
img